In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
class input_embedding(nn.Module):
  def __init__(self,d_model: int, vocab_size: int):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)
  def forward(self, x):
    return self.embedding(x)*math.sqrt(self.d_model)

In [ ]:
class positional_encoding(nn.Module):
  def __init__(self, d_model: int,seq_len: int, dropout: float):
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    pe = torch.zeros(seq_len, d_model)

    position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp((torch.arrange(0,d_model,2).float() * (-math.log(10000.0)/d_model)))

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    self.register_buffer('pe',pe)

  def forward(self, x):
    x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
    return self.dropout(x)

In [ ]:
class Layer_Normalization(nn.Module):
  def __init__(self, eps: float = 10**-6)-> None:
    super().__init__()
    self.eps = eps
    self.alphas = nn.Parameter(torch.ones(1))
    self.bias = nn.Parameter(torch.zero(1))

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim = True)
    std = x.std(dim = -1, keepdim = True)
    return  self.alphas * (x - mean) / (std + self.eps) + self.bias

In [ ]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model: int, d_ff: int, dropout: float) ->None:
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)
  def forward(self, x):
    return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

In [ ]:
class MultiHeadAttentionBlock(nn.Module):
  def __init__(self, d_model: int, h: int, dropout: int)-> None:
    super().__init__()
    self.d_model = d_model
    self.h = h
    assert d_model %  h == 0, 'd_model is not dividible by h'

    self.d_k = d_model // h
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    self.w_o = nn.Linear(d_model, d_model)
    self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
      d_k = query.shape[-1]
      attention_scores = (query @ key.transpose(-2,-1)) / math.sqrt(d_k)
      if mask is not None:
        attention_scores.masked_fill(mask == 0, 1e-9)
      attention_scores = attention_scores.softmax(dim = -1)
      if dropout is not None:
        attention_scores = dropout(attention_scores)
      return (attention_scores @ value), attention_scores
    def forward(self, q, k, v, mask):
      query = self.w_q(q)
      key = self.w_k(k)
      value = self.w_v(v)


      query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)
      key = query.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)
      value = value.view(value.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)

      x, self.attention_scores = MultiHeadAttentionBlock(query, key, value, mask, self.dropout)

      x = x.transpose(1,2).contiguous.view(x.shape[0], -1, self.h, self.d_k)

      return self.w_o(x)

In [ ]:
class ResidualConnection(nn.Module):
  def __init__(self, dropout: float) ->None:
    super().__init__()
    nn.dropout = nn.Dropout(dropout)
    nn.norm = Layer_Normalization()

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderBlock(nn.Module):
  def __init__(self, self_attention: MultiHeadAttentionBlock, feed_forward: FeedForwardBlock, dropout: float) ->None:
    super().__init__()
    self.self_attention = self_attention
    self.feed_forward  = feed_forward
    self.residual_connection = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

  def forward(self, x, src_mask):
    x = self.residual_connection[0](x, lambda x: self.self_attention(x, x, x, src_mask))
    x = self.residual_connection[1](x, self.feed_forward)
    return x

In [ ]:
class Encoder(nn.Module):
  def __init__(self, layers: nn.ModuleList)->None:
    super().__init__()
    self.layers = layers
    self.norm = Layer_Normalization()

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

In [ ]:
class DecoderBlock(nn.Module):
  def __init__(self, self_attention: MultiHeadAttentionBlock, cross_attention: MultiHeadAttentionBlock, feed_forward: FeedForwardBlock, dropout: float):
    super().__init__()
    self.self_attention = self_attention
    self.cross_attention = cross_attention
    self.feed_forward = feed_forward
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    super().__init__()
    x = self.residual_connections[0](x, lambda x: self.self_attention(x,x,x,tgt_mask))
    x = self.residual_connections[1](x, lambda x: self.self_attention(x,encoder_output, encoder_output, src_mask))
    x = self.residual_connections[2](x, lambda x: self.self_attention(x, self.feed_forward))
    return x

In [ ]:
class Decoder(nn.Module):
  def __init__(self, layers: nn.ModuleList) -> None:
    self.layers = layers
    self.norm = Layer_Normalization()

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    for layer in self.layers:
      x = layer(x, encoder_output, src_mask, tgt_mask)
      return self.norm(x)

In [ ]:
class ProjectionLayer(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def __init__(self, x):
    return torch.log_softmax(self.proj(x), dim = -1)

In [ ]:
class Transformer(nn.Module):
  def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: input_embedding, tgt_embed: input_embedding, src_pos: positional_encoding, tgt_pos: positional_encoding, projection_layer: ProjectionLayer):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encoder(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decoder(self, tgt, encoder_output, src_mask, tgt_mask):
    src = self.tgt_embed(tgt)
    src = self.tgt_pos(src)
    return self.decoder(src, encoder_output, src_mask, tgt_mask)

  def project(self, x):
    return self.projection_layer(x)


In [ ]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048)-> Transformer:
  src_embed = input_embedding(d_model, src_vocab_size)
  tgt_embed = input_embedding(d_model, tgt_vocab_size)

  src_pos = positional_encoding(d_model, src_seq_len, dropout)
  tgt_pos = positional_encoding(d_model, tgt_seq_len, dropout)

  encoder_blocks = []
  for _ in range(N):
    encoder_self_attention = MultiHeadAttentionBlock(d_model, h, dropout)
    feed_forward = FeedForwardBlock(d_model, d_ff, dropout)
    encoder_block = encoder_block(encoder_self_attention, feed_forward, dropout)
    encoder_blocks.append(encoder_block)

  decoder_blocks = []
  for _ in range(N):
    decoder_self_attention = MultiHeadAttentionBlock(d_model, h, dropout)
    decoder_cross_attention = MultiHeadAttentionBlock(d_model, h, dropout)
    feed_forward = FeedForwardBlock(d_model, d_ff, dropout)
    decoder_block = DecoderBlock(decoder_self_attention, decoder_cross_attention, feed_forward, dropout)
    decoder_blocks.append(decoder_block)

  encoder = Encoder(nn.ModuleList(encoder_blocks))
  decoder = Decoder(nn.ModuleList(decoder_blocks))

  projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

  transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

  for p in transformer.parameters:
    if p.dim > 1:
      nn.init.xavier_uniform_(p)

  return transformer